In [18]:
import nltk
from nltk.corpus import brown
from nltk.data import find
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('data/NN_Data.csv').drop(columns=['Unnamed: 0'])
df["no_punct"] = df['text'].str.replace('[^\w\s]','')
df['str_list'] = df.no_punct.apply(lambda s: s.split(' '))

In [5]:
w2v = Word2Vec(df['str_list'].tolist(), min_count=1, vector_size = 300,workers=3, window =3, sg = 1)

In [6]:
kv = m.wv

In [9]:
EMBEDDING_DIM = len(kv['heart'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = kv.vectors
vocab_dict = kv.key_to_index

In [10]:
MAX_SEQUENCE_LENGTH = 5  # Keras' embedding layer expects a specific input length. Padding is often needed here.

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [11]:
try:
    del tf_model
except:
    pass

In [12]:
tf_model = tf.keras.Sequential()

tf_model.add(embedding_layer)                                        # embedding layer

tf_model.add(tf.keras.layers.Conv1D(
    filters=10, 
    kernel_size=3, 
    strides=1, 
    padding='same', 
    activation='relu', 
    use_bias=True,
    kernel_initializer='glorot_uniform', 
    bias_initializer='zeros')
            )    

tf_model.add(tf.keras.layers.GlobalMaxPooling1D())


tf_model.add(Dense(100, activation='relu'))                          # hidden layer
tf_model.add(Dense(1, activation='sigmoid'))                         # classification layer

In [60]:
df[["str_list", "id_list"]]

,str_list,id_list
0,"[My, dad, had, a, heart, attack, and, spent, o...","[[75], [149], [24], [4], [52], [107], [2], [64..."
1,"[I, had, dangerous, open, heart, surgery, due,...","[[0], [24], [2967], [374], [52], [440], [447],..."
2,"[In, June, of, 2020, my, Dad, collapsed, in, o...","[[372], [3523], [5], [9391], [6], [801], [2421..."
3,"[after, my, twin, aunts, were, born, my, grand...","[[104], [6], [6215], [2833], [74], [1373], [6]..."
4,"[Not, me, but, my, mother, is, the, most, seve...","[[382], [16], [18], [6], [274], [14], [3], [13..."
...,...,...
2783,"[My, Dad, and, brother, dont, care, as, long, ...","[[75], [801], [2], [405], [44], [181], [25], [..."
2784,"[Im, throwing, it, out, so, Im, not, worried, ...","[[36], [2193], [10], [38], [27], [36], [26], [..."
2785,"[I, had, a, terribly, emotionally, abusive, mo...","[[0], [24], [4], [3846], [896], [2498], [274],..."
2786,"[Hi\n\nIm, back, in, a, relationship, and, the...","[[8692], [87], [9], [4], [205], [2], [3], [236..."


In [137]:
def sents_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    text_ids = [vocab_dict[word] for word in sentences]
    return np.asarray(text_ids).astype(np.int64)

In [142]:
df["id_list"] = df["str_list"].apply(sents_to_ids)
df["outcome_2d"] = df["outcome"].apply(lambda x: np.asarray([x]).astype(np.int64))
X = np.array(df["id_list"])
y = np.array(df["outcome_2d"])

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [144]:
y_train

array([array([0]), array([1]), array([0]), ..., array([1]), array([1]),
       array([1])], dtype=object)

In [145]:
tf_model.compile(optimizer='adam', loss='binary_crossentropy')


In [146]:
tf_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, verbose=0)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [126]:
y_train

array([array([0]), array([1]), array([0]), ..., array([1]), array([1]),
       array([1])], dtype=object)

In [98]:
np.asarray(X_train).astype('int')

ValueError: setting an array element with a sequence.

In [100]:
[print(i.shape, i.dtype) for i in tf_model.inputs]
[print(o.shape, o.dtype) for o in tf_model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in tf_model.layers]

(None, 5) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
embedding (None, 5) float32
conv1d (None, 5, 300) float32
global_max_pooling1d (None, 5, 10) float32
dense (None, 10) float32
dense_1 (None, 100) float32


[None, None, None, None, None]

In [103]:
#print(tf_model.predict(X_train))
type(X_train)

numpy.ndarray

In [104]:
type(X_train[0])

numpy.ndarray

In [108]:
type(y_train)

numpy.ndarray

In [106]:
X_tensor = tf.convert_to_tensor(X_train[0], dtype=tf.int64) 